<a href="https://colab.research.google.com/github/qkhadim22/PatuCrane_Exudyn/blob/main/MainScript_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1: Import necessary libraries


In [6]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import csv



Step 1.1: Path to saving and loading FF_NN model data.

In [7]:
from google.colab import drive

LiftFlexible = True
PatuFlexible = False
LoadSaved_NN = False


if LiftFlexible:
    # Path to NN model
    keras_model_path  = '/content/PatuCrane_Exudyn/ExData'
    SimData           = np.genfromtxt('/content/PatuCrane_Exudyn/ExData/LiftFlexible/LiftBoomData.csv',
                                      delimiter=",",dtype=np.float32)

elif PatuFlexible:
    # Path to NN model
    keras_model_path  = '/content/PatuCrane_Exudyn/ExData/NN_Data/PatuFlexible_NN'
    SimData           = np.genfromtxt('ExData/LiftFlexible/PatuFlexible.csv',
                                      delimiter=",",dtype=np.float32)

else:
    # Path to NN model
    keras_model_path  = '/content/PatuCrane_Exudyn/ExData/NN_Data/PatuRigid_NN'
    SimData           = np.genfromtxt('ExData/LiftFlexible/PatuRigid.csv',
                                      delimiter=",",dtype=np.float32)


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Step 2: Loading and preparing the data

Step 2.1: Load previous keras model or import new dataset, and split into train and test data

In [ ]:
if LoadSaved_NN:
  restored_keras_model = tf.keras.models.load_model(keras_model_path)
else:
  n_samples, n_features = SimData.shape
  n_features           -= 1
  X                     = SimData[:,0:n_features]
  Y                     = SimData[:,n_features]

  print('Shape of Lift Boom data =',SimData.shape)
  print('Number of samples =',n_samples)
  print('Number of features =',n_features)
  print(X.shape, Y.shape)

Step 2.2: Splitting data into training and testing sets

In [28]:
  from sklearn.model_selection import train_test_split
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=6)
  Y_train = Y_train.reshape(-1, 1)
  Y_test = Y_test.reshape(-1, 1)
  print('Length of training dataset=',len(X_train))
  print('Length of testing dataset=',len(X_test))
  print('Shape of testing dataset=',X_test.shape)

Length of training dataset= 1920
Length of testing dataset= 481
Shape of testing dataset= (481, 6)


#Step 3: Create the model (Define the network architecture using keras).

In [ ]:
 input_dim  = n_features
 output_dim = 1
 model      = tf.keras.Sequential([tf.keras.layers.Dense(6, activation='relu', input_shape=(input_dim,)),
                                   tf.keras.layers.Dense(128, activation='relu'),
                                   tf.keras.layers.Dense(64, activation='relu'),
                                   tf.keras.layers.Dense(output_dim)])
 model.summary()

#Step 4: Compiling and training the model (Train the model using sgd).

In [2]:
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.06),
              loss='mean_squared_error',
              metrics=['accuracy'])

  history=model.fit(X_train,Y_train,validation_data=(X_test,Y_test), epochs=10)

NameError: ignored

#Step 5: Evaluate the network and make predictions (Testing Performance).

In [ ]:
test_loss, test_acc = model.evaluate(X_test,Y_test)
print("Loss=%.3f" %test_loss)
print("Accuracy=%.3f" %test_acc)

n = random.randint(0,9999)
plt.imshow(X_test[n])
plt.show()

predicted_value=model.predict(X_test)
print("Handwritten number in the image is= %d" %np.argmax(predicted_value[n]))

#Step 6: Training loss and accuracy.

In [ ]:
    history.history.keys()

    #Figure 1
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train','Validation'],loc='upper left')
    plt.show()


    #Figure 2
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train','Validation'],loc='upper left')
    plt.show()


    #Figure 3
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Training Loss And Accuracy')
    plt.ylabel('Accuracy/Loss')
    plt.xlabel('Epoch')
    plt.legend(['Accuracy','Val Accuracy','Loss','Val Loss'],loc='upper left')
    plt.show()

#Step 7: Save model

In [37]:
 model.save(keras_model_path)